# Trabalho 2

### Sobre o trabalho
- O trabalho tem os seguintes objetivos:
    - Estudar a tarefa de **POS Tagging** (Part-Of-Speech (POS) Tagging) para a língua Portuguesa
    - Classificar a classe gramatical de palavras em Português
    - Implementar e avaliar a precisão de um modelo de POS tagging

### Resumo do trabalho
- O modelo utilizado foi o BERT
- Foi feito um finetuning no modelo, para classificação de classe gramatical

### Ferramentas, dados e bibliotecas utilizadas
- O corpus de treino, avaliação e teste utilizado foi o corpus recomendado na descrição do trabalho: macmorpho
- A principal biblioteca utilizada para manipular os dados foi a biblioteca recomendada: `transformers`
  - Outras bibliotecas úteis utilizadas: `torch`, `numpy` e `scikit-learn`

Trabalho Prático II

Nosso objetivo é estudar a tarefa de POS Tagging para a língua Portuguesa. Para isso utilizaremos o corpus Mac-Morpho, produzido pelo grupo NILC da ICMC USP. O Mac-Morpho oferece dados para treinamento, validação e teste de modelos preditivos, capazes de classificar a classe gramatical de palavras em Português. Para acessar o conjunto de classes, acesse http://nilc.icmc.usp.br/macmorpho/macmorpho-manual.pdf.


O corpus está disponível em http://nilc.icmc.usp.br/macmorpho/macmorpho-v3.tgz. Você deverá implementar e avaliar a precisão de um modelo de POS tagging, a sua escolha. Voce pode utilizar pacotes que facilitem a implementação, como gensim, e transformers. Você deverá entregar documentação embutida em um notebook, detalhando a tarefa, a metodologia e os resultados. Sua análise deverá discutir quais as classes gramaticais para as quais a precisão é maior/menor. Não utilizaremos LLMs para essa tarefa, mas a sugestão é utilizar Transformers disponiveis e pre-treinados, em especial o BERT.

Exemplo de dados de treino:
```
Jersei_N atinge_V média_N de_PREP Cr$_CUR 1,4_NUM milhão_N na_PREP+ART venda_N da_PREP+ART Pinhal_NPROP em_PREP São_NPROP Paulo_NPROP ._PU Programe_V sua_PROADJ viagem_N à_PREP+ART Exposição_NPROP Nacional_NPROP do_NPROP Zebu_NPROP ,_PU que_PRO-KS começa_V dia_N 25_N ._PU
```

# Discussão

#### Geral
- Quantas sentenças no treino, validação e teste?
- Quantas tags diferentes? Quantos parametros treináveis tem o modelo no total?

#### Específico para avaliação do trabalho
- `Implementar e avaliar a precisão:` isso pra mim é falar de acurácia.
- `detalhando a tarefa, metodologia e os resultados`
- `discutir quais classes gramaticais tem maior/menor precisão`

In [1]:
# %pip install transformers[torch]
%pip install accelerate -U --quiet

## Carregar e tokenizar os dados de treino

In [2]:
from transformers import BertTokenizerFast, BertForTokenClassification, TrainingArguments, Trainer
from sklearn.preprocessing import LabelEncoder
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import numpy as np
import torch
from torch.utils.data import Dataset

# Create a device object
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the BERT tokenizer
bert_tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

# Initialize the label encoder
label_encoder = LabelEncoder()

# Set max_length
max_length = 512

def preprocess_and_tokenize(data):
    sentences = []
    pos_tags = []

    # Split data into sentences and POS tags
    for line in tqdm(data, desc="Splitting data into sentences and POS tags"):
        words = []
        tags = []
        for word in line.split():
            split_word = word.split('_')
            words.append(split_word[0])
            tags.append(split_word[1])
        sentences.append(words)
        pos_tags.append(tags)

    # Tokenize the sentences
    tokenized_inputs = bert_tokenizer(sentences, truncation=True, padding='max_length',
                                      max_length=max_length, is_split_into_words=True)

    # Pad input_ids to max_length and convert to a numpy array
    input_ids = pad_sequence([torch.tensor(i) for i in tokenized_inputs["input_ids"]], batch_first=True)
    # input_ids = input_ids.numpy()
    input_ids = input_ids.to(device)  # Move to GPU
    print(f"Shape of input_ids: {input_ids.shape}")

    # Handle the POS tags
    new_pos_tags = []
    for sent_tags, input_id in zip(pos_tags, tokenized_inputs["input_ids"]):
        new_tags = []
        for tag in sent_tags:
            new_tags.extend([tag] * len(input_id))
        new_pos_tags.append(new_tags[:len(input_id)])

    # Encode the POS tags
    encoded_pos_tags = [label_encoder.fit_transform(tags) for tags in new_pos_tags]

    # Pad the POS tags
    padded_pos_tags = pad_sequence([torch.tensor(tags) for tags in encoded_pos_tags], batch_first=True)
    # padded_pos_tags = padded_pos_tags.numpy()
    padded_pos_tags = padded_pos_tags.to(device)  # Move to GPU

    # Create attention masks
    attention_masks = [[float(i != 0.0) for i in seq] for seq in input_ids]
    attention_masks = torch.tensor(attention_masks).to(device)  # Move to GPU
    # print(f"Shape of tokenized inputs: {tokenized_inputs['input_ids'].shape}")
    # return tokenized_inputs["input_ids"], padded_pos_tags, attention_masks
    return input_ids, padded_pos_tags, attention_masks

### Mount Google drive and change active directory

In [3]:
from google.colab import drive
drive.mount('/content/drive')

# change execution folder to MyDrive/UFMG/NLP/
import os
os.chdir('drive/MyDrive/UFMG/NLP/')
# os.getcwd()
# os.listdir()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
with open('data/macmorpho-train.txt', 'r') as f:
    data = f.readlines()
    input_ids, tags, masks = preprocess_and_tokenize(data)

with open('data/macmorpho-dev.txt', 'r') as f:
    data = f.readlines()
    val_input_ids, val_tags, val_masks = preprocess_and_tokenize(data)

Splitting data into sentences and POS tags: 100%|██████████| 37948/37948 [00:00<00:00, 81170.50it/s]


Shape of input_ids: torch.Size([37948, 512])


Splitting data into sentences and POS tags: 100%|██████████| 1997/1997 [00:00<00:00, 95580.72it/s]


Shape of input_ids: torch.Size([1997, 512])


In [5]:
input_ids.shape, tags.shape, len(masks)
# len(val_input_ids), val_tags.shape, len(val_masks)

(torch.Size([37948, 512]), torch.Size([37948, 512]), 37948)

## Finetuning do modelo

#### Modelo BERT `bert-base-multilingual-cased`
- 179M parâmetros treináveis
- Modelo pré-treinado em 104 idiomas, incluindo o português
- Modo de treinamento: Finetuning apenas na última camada

In [9]:
# Define a custom PyTorch Dataset
class CustomDataset(Dataset):
    def __init__(self, input_ids, masks, tags):
        self.input_ids = input_ids
        self.masks = masks
        self.tags = tags.to('cpu')

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.masks[idx],
            'labels': self.tags[idx]
        }

# Move the tensors to the CPU
input_ids_cpu = input_ids.to('cpu')
masks_cpu = masks.to('cpu')
tags_cpu = tags.to('cpu')
val_input_ids_cpu = val_input_ids.to('cpu')
val_masks_cpu = val_masks.to('cpu')
val_tags_cpu = val_tags.to('cpu')

# Convert training data into PyTorch Dataset
# train_dataset = CustomDataset(input_ids, masks, tags)
train_dataset = CustomDataset(input_ids_cpu, masks_cpu, tags_cpu)

# Convert validation data into PyTorch Dataset
# eval_dataset = CustomDataset(val_input_ids, val_masks, val_tags)
eval_dataset = CustomDataset(val_input_ids_cpu, val_masks_cpu, val_tags_cpu)

# Initialize the BERT model
model = BertForTokenClassification.from_pretrained(
    'bert-base-multilingual-cased',
    num_labels=512,
)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4, # 16
    per_device_eval_batch_size=4,  # 64
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    # device=device,
    # accelerator='cpu'
    # bf16=True, # or fp16, to reduce memory/computer usage/requirements
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# print(model)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [24]:
classification_layer = model.classifier
print(f"classification_layer: {classification_layer}")

# Freeze all the layers of the BERT model
for param in model.bert.parameters():
    param.requires_grad = False

# Train only the classifier layer
for param in model.classifier.parameters():
    param.requires_grad = True

# for name, param in model.named_parameters():
#     print(name, param.requires_grad)

# Check that only the classifier layer is trainable
for name, param in model.named_parameters():
    if 'classifier' in name:  # Check if the parameter belongs to the classifier layer
        assert param.requires_grad == True  # Assert that the classifier layer parameters are trainable
    else:
        assert param.requires_grad == False  # Assert that all other parameters are not trainable

Linear(in_features=768, out_features=512, bias=True)
bert.embeddings.word_embeddings.weight False
bert.embeddings.position_embeddings.weight False
bert.embeddings.token_type_embeddings.weight False
bert.embeddings.LayerNorm.weight False
bert.embeddings.LayerNorm.bias False
bert.encoder.layer.0.attention.self.query.weight False
bert.encoder.layer.0.attention.self.query.bias False
bert.encoder.layer.0.attention.self.key.weight False
bert.encoder.layer.0.attention.self.key.bias False
bert.encoder.layer.0.attention.self.value.weight False
bert.encoder.layer.0.attention.self.value.bias False
bert.encoder.layer.0.attention.output.dense.weight False
bert.encoder.layer.0.attention.output.dense.bias False
bert.encoder.layer.0.attention.output.LayerNorm.weight False
bert.encoder.layer.0.attention.output.LayerNorm.bias False
bert.encoder.layer.0.intermediate.dense.weight False
bert.encoder.layer.0.intermediate.dense.bias False
bert.encoder.layer.0.output.dense.weight False
bert.encoder.layer.0.ou

In [ ]:
# Train the model
trainer.train()

Step,Training Loss
500,5.195500
1000,1.468400
1500,0.101100
2000,0.034500
2500,0.019400
3000,0.012600
3500,0.009800
4000,0.006300
4500,0.004300
5000,0.003000


In [ ]:
# %pip list transfomers

### Load test data

In [ ]:
with open('data/macmorpho-test.txt', 'r') as f:
    data = f.readlines()
    test_input_ids, test_tags, test_masks = preprocess_and_tokenize(data)

# Referências
- [Hugging Face: Transformers - BERT](https://huggingface.co/docs/transformers/model_doc/bert)
- [Hugging Face: BERT multilingual base model (cased)](https://huggingface.co/bert-base-multilingual-cased)